In [1]:
!pip install pyarrow

In [2]:
pip install --upgrade requests urllib3 certifi

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# загружаю набор данных с историческим курсом валют

df1 = pd.read_parquet(r'C:\Users\Алексей\Downloads\historical_currency_exchange.parquet')

In [6]:
# загружаю набор данных с финансовыми транзакциями

df2 = pd.read_parquet(r'C:\Users\Алексей\Downloads\transaction_fraud_data.parquet')

In [10]:
# получаю базовый обзор размера

print(f"Количество строк df1: {df1.shape[0]}")
print(f"Количество столбцов df1: {len(df1.columns)}")

print(f"Количество строк df2: {df2.shape[0]}")
print(f"Количество столбцов df2: {len(df2.columns)}")

Количество строк df1: 31
Количество столбцов df1: 12
Количество строк df2: 7483766
Количество столбцов df2: 23


In [12]:
# создаю случайную подвыборку, чтобы на ней понять структуру, распределение и аномалии в данных

sample_df2 = df2.sample(frac=0.05, random_state=42) # 5% случайных строк

In [13]:
# получаю информацию о подвыборке

sample_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374188 entries, 543612 to 7429067
Data columns (total 23 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   transaction_id           374188 non-null  object        
 1   customer_id              374188 non-null  object        
 2   card_number              374188 non-null  int64         
 3   timestamp                374188 non-null  datetime64[ns]
 4   vendor_category          374188 non-null  object        
 5   vendor_type              374188 non-null  object        
 6   vendor                   374188 non-null  object        
 7   amount                   374188 non-null  float64       
 8   currency                 374188 non-null  object        
 9   country                  374188 non-null  object        
 10  city                     374188 non-null  object        
 11  city_size                374188 non-null  object        
 12  card_type 

In [ ]:
# описательная статистика для признаков

sample_df2.describe(include='all')

In [ ]:
# определяю кол-во мошенеческих транзакций

fraud_counts = sample_df2['is_fraud'].value_counts()
print("\nРаспределение метки мошенничества:")
print(fraud_counts)

In [ ]:
# вывожу топ-10 самых частотных значений для каждой из категорий — vendor_category, card_type, country

for col in ['vendor_category', 'card_type', 'country']:
    print(f"\nТоп-10 значений для {col}:")
    vc = sample_df2[col].value_counts().head(10)
    print(vc)
    plt.figure(figsize=(10,4))
    sns.barplot(x=vc.index, y=vc.values)
    plt.title(f'Топ-10 значений в {col}')
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
# провожу временной анализ — количество транзакций по дате/часу
sample_df2['transaction_hour'] = sample_df2['timestamp'].dt.hour
hourly_counts = sample_df2.groupby('transaction_hour').size()

plt.figure(figsize=(10,4))
sns.barplot(x=hourly_counts.index, y=hourly_counts.values)
plt.title('Количество транзакций по часам суток')
plt.xlabel('Час')
plt.ylabel('Количество транзакций')
plt.show()

In [ ]:
# провожу анализ суммы транзакций — распределение и выбросы (на выборке)
plt.figure(figsize=(10,5))
sns.histplot(sample_df2['amount'], bins=50, kde=True)
plt.title('Распределение суммы транзакций (выборка)')
plt.xlabel('Сумма транзакции')
plt.ylabel('Количество')
plt.show()

In [ ]:
# проверяю гипотезу 1 -- несоответствие валют

# для каждой страны нахожу самую частую валюту
home_currency_map = (
    sample_df2.groupby('country')['currency']
    .agg(lambda x: x.value_counts().index[0])  # мода
    .to_dict()
)

# добавляю колонку с "домашней" валютой
sample_df2['home_currency'] = sample_df2['country'].map(home_currency_map)

# признак несоотвествия валют
sample_df2['currency_mismatch'] = sample_df2['currency'] != sample_df2['home_currency']

# доля fraud при совпадении и несовпадении валют
hyp1 = sample_d2f.groupby('currency_mismatch')['is_fraud'].mean()
print("\n1. Доля fraud при несовпадении валют:")
print(hyp1)


In [ ]:
# проверяю гипотезу 2 -- мошенники чаще используют веб-канал, чем мобильный 
hyp2 = sample_d2f.groupby('channel')['is_fraud'].mean()
print("\n2. Доля fraud по каналам:")
print(hyp2)

In [ ]:
# проверяю гипотезу 3 -- Fraud чаще происходит при транзакциях вне страны клиента 
hyp3 = sample_d2f.groupby('is_outside_home_country')['is_fraud'].mean()
print("\n3. Доля fraud при транзакциях вне страны клиента:")
print(hyp3)

In [ ]:
# проверяю гипотезу 4 -- В выходные доля fraud выше, чем в будни 
hyp4 = sample_df2.groupby('is_weekend')['is_fraud'].mean()
print("\n4. Доля fraud в выходные и будни:")
print(hyp4)

In [ ]:
# проверяю гипотезу 5 -- Оплата у high-risk вендоров даёт в N раз выше шанс fraud 
hyp5 = sample_df2.groupby('is_high_risk_vendor')['is_fraud'].mean()
print("\n5. Доля fraud у high-risk и обычных вендоров:")
print(hyp5)

# Вычисляем отношение вероятностей fraud (во сколько раз выше)
ratio = hyp5[True] / hyp5[False]
print(f"\nВо сколько раз шанс fraud выше у high-risk вендоров: {ratio:.2f}x")